In [ ]:
import sys,os
venv_path = os.path.join(os.path.dirname(os.getcwd()), 'venv')
if os.path.exists(venv_path):
    # Add the venv to the path
    venv_site_packages = os.path.join(venv_path, 'Lib', 'site-packages')
    if venv_site_packages not in sys.path:
        sys.path.insert(0, venv_site_packages)
    print(f"Added {venv_site_packages} to Python path")
else:
    print("Virtual environment not found")

In [ ]:
import pandas as pd
import os
base_dir_path=os.path.dirname(os.getcwd())
base_dir_path

In [15]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [26]:
import os
import pandas as pd
import faiss
from src.embed_dataset import GenerateEmbeddings
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

sampled_datavals_product=pd.read_parquet(os.path.join(base_dir_path,"dataset/sampled_dataset/sampled_datavals_product_data.parquet"))
sampled_datavals_product_imbalanced=pd.read_parquet(os.path.join(base_dir_path,"dataset/sampled_dataset/sampled_datavals_product_data_imbalanced.parquet"))

sampled_queries = pd.read_parquet(os.path.join(base_dir_path,"dataset/sampled_dataset/sampled_query_examples.parquet"))


dataset_examples=pd.read_parquet(os.path.join(base_dir_path,"dataset/source_files/shopping_queries_dataset_examples.parquet"))
filt_examples=dataset_examples[(dataset_examples["product_locale"]=="us") & (dataset_examples["esci_label"]=="E")]

class ExperimentIndex:
        def __init__(self, index_path, embeddings_obj) -> None:
            self.index_path = index_path
            self.index = self.load_index(self.index_path)
            self.embeddings_obj = embeddings_obj

        def load_index(self,index_path):
            return faiss.read_index(index_path)
        
        def experiment_with_index(self,query_val, top_k):
            query_embedding = self.embeddings_obj.encode_query(query_val)
            relevant_examples_df=filt_examples[filt_examples["query"]==query_val]
            relevant_ids=relevant_examples_df["product_id"].tolist()

            scores, indices = self.index.search(query_embedding, top_k)
            retrieved_examples=sampled_datavals_product[sampled_datavals_product['ID'].isin(indices[0])]
            relevant_examples_df=filt_examples[filt_examples["query"]== query_val]
            relevant_ids=relevant_examples_df["product_id"].tolist()
            
            return scores,retrieved_examples, relevant_examples_df
        

if __name__ == "__main__":
    generate_embeddings_base_obj=GenerateEmbeddings(model_name="intfloat/multilingual-e5-small")

    experiment_index_obj=ExperimentIndex(os.path.join(base_dir_path,"vector_indexes/balanced_dataset/384dim/product_fields_exclude_desc_384dim_hnsw.index"), generate_embeddings_base_obj)
    scores,retrieved_examples, relevant_examples_df=experiment_index_obj.experiment_with_index("vena iphone case",5)
    relevant_examples_df=filt_examples[filt_examples["query"]=="vena iphone case"]

    print(scores)
    print(retrieved_examples)
    print(relevant_examples_df)

   
    

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-small
INFO:src.embed_dataset:Model loaded. Embedding dimension: 384
Batches: 100%|██████████| 1/1 [00:00<00:00, 24.69it/s]


[[0.2601393  0.26462764 0.26757807 0.27034277 0.27728617]]
         product_id                                      product_title  \
753861   B018ES4D20  iPhone 6S Plus Wallet Case, Vena vCommute Drop...   
753868   B07VSSSWRJ  Vena vLove Glitter Clear Case Compatible with ...   
1360552  B01KUFDYCA  Vena vCommute Wallet Case Compatible with Appl...   
1360553  B01KUFTXUM  Vena Wallet Case Compatible with iPhone 8 Plus...   
1360560  B07V1XXWCG  Vena vCommute Wallet Case Compatible with Appl...   

                                       product_description  \
753861                                                None   
753868   Slim and Protective iPhone 11 Glitter Case Wit...   
1360552                                               None   
1360553                                               None   
1360560  Patented iPhone 11 Wallet Case With Vena's pat...   

                                      product_bullet_point product_brand  \
753861   [PATENTED design] Compatible with: App

In [23]:
sampled_queries.iloc[0]

example_id                 2111746
query             vena iphone case
query_id                    108170
product_id              B078HZZF41
product_locale                  us
esci_label                       E
small_version                    0
large_version                    1
split                        train
Name: 2111746, dtype: object